# Написание класса который возвращает случайную выборку объектов и случайное подпространство признаков

In [328]:
import numpy as np

class sample(object):
  def __init__(self, X, n_subspace,random_state):
    self.random_state = random_state
    self.idx_subspace = self.random_subspace(X, n_subspace,random_state)

  def __call__(self, X, y):
    idx_obj = self.bootstrap_sample(X)
    X_sampled, y_sampled = self.get_subsample(X, y, self.idx_subspace, idx_obj)
    return X_sampled, y_sampled

  @staticmethod
  def bootstrap_sample(X, random_state):
    """
    Заполните тело этой функции таким образом, чтобы она возвращала массив индексов выбранных при помощи бэггинга индексов.
    Пользуйтесь только инструментами, реализованными в numpy.random, выставляя везде, где это необходимо, random_state=42
    """
    np.random.seed = random_state
    idx = []
    for i in range(X.shape[0]):
      idx.append(np.random.choice(np.arange(0,X.shape[0])))
    return np.unique(idx)
  @staticmethod
  def random_subspace(X, n_subspace, random_state):
    """
    Заполните тело этой функции таким образом, чтобы она возвращала массив индексов выбранных при помощи метода случайных подпространств признаков
    Количество этих признаков передается при помощи аргумента n_subspace
    Пользуйтесь только инструментами, реализованными в numpy.random, выставляя везде, где это необходимо, random_state=42
    """
    np.random.seed = random_state
    answer = []
    idx = np.arange(0,X.shape[1])
    answer = np.random.choice(idx,replace= False,size = n_subspace)
    return answer

  @staticmethod
  def get_subsample(X, y, idx_subspace, idx_obj):
    """
    Заполните тело этой функции таким образом, чтобы она возвращала подвыборку x_sampled, y_sampled
    по значениям индексов признаков(idx_subspace) и объектов(idx_obj) , которые должны в неё попасть
    """
    X_sampled = X[np.ix_(idx_obj,idx_subspace)]
    y_sampled = y[idx_obj]
    return X_sampled, y_sampled


In [142]:
x= np.array([[1,2,3], [4,5,6], [7,8,9]])
Y = np.array([1, 2, 3])
s = sample(x, 2)
bootstrap_indices = s.bootstrap_sample(x)
X_sampled, y_sampled = s.get_subsample(x, Y, s.idx_subspace, bootstrap_indices)

In [317]:
bootstrap_indices

array([0, 2])

In [115]:
X_sampled

array([[2, 3]])

In [113]:
s.idx_subspace

array([1, 2])

# Написание класса random_forest. Поиск наилучших параметров для задачи Ирисов Фишера

In [368]:
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

N_ESTIMATORS = 3
MAX_DEPTH = 12
SUBSPACE_DIM = 3
np.random.seed = 42
def vote(predicted:np.array)->np.array:
    answer = []
    for row in predicted:
        result,votes = np.unique(row,return_counts=True)
        answer.append(result[np.argmax(votes)])
    return np.array(answer).reshape(-1,1)


class random_forest:
    def __init__(self, 
                 n_estimators: int,
                 max_depth: int,
                 subspaces_dim: int,
                 random_state: float) -> None:
        self.n_estimators = n_estimators
        self.max_depth = max_depth
        self.subspaces_dim = subspaces_dim
        self.random_state = random_state
        self._estimators = {}
        self.subspace_idx =  {}
    
    def fit(self, X: np.array, y: np.array) -> None:
         #Обучаем n_estimators моделей на разных подвыборках данных.
        i = 0
        for i in range(self.n_estimators):
            random_state = self.random_state + i
            data = sample(X, self.subspaces_dim,random_state = random_state)
            bootstrap_indices = data.bootstrap_sample(X,random_state=random_state)
            x_sampled, y_sampled = data.get_subsample(X, y, data.idx_subspace, bootstrap_indices)
            model = DecisionTreeClassifier(max_depth=self.max_depth, random_state=self.random_state)
            model.fit(x_sampled,y_sampled)
            self._estimators[i] = model 
            self.subspace_idx[i] = data.idx_subspace


    def predict(self, X: np.array) -> np.array: # Методом голосования определяем ответ для каждого объекта.
        def _predict(i: int):
            return self._estimators[i].predict(X[:, self.subspace_idx[i]]).reshape(-1,1)
        
        answer = _predict(0)
        for i in range(1, self.n_estimators):
            answer = np.hstack((answer, _predict(i))) #Получаем матрицу ответов всех моделей.
        
        return vote(answer)
                


In [366]:
X,y = load_iris(return_X_y=True)
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3)
print(X_test.shape)

(45, 4)


In [369]:
result = {}
for i in range(1,20):
    for j in range(1,20):
        for k in range(1,5):
            rf = random_forest(i, j, k, 42)
            rf.fit(X_train, y_train)
            result[str(i)+' '+str(j)+' '+str(k)] = accuracy_score(y_test,rf.predict(X_test))


In [363]:

sorted(result.items(),key = lambda x:x[1],reverse= True)



[('3 12 3', 1.0),
 ('1 11 1', 0.9777777777777777),
 ('1 11 3', 0.9777777777777777),
 ('3 11 3', 0.9777777777777777),
 ('3 13 2', 0.9777777777777777),
 ('6 1 3', 0.9777777777777777),
 ('10 2 2', 0.9777777777777777),
 ('11 1 4', 0.9777777777777777),
 ('14 2 2', 0.9777777777777777),
 ('1 11 2', 0.9555555555555556),
 ('1 19 2', 0.9555555555555556),
 ('3 9 2', 0.9555555555555556),
 ('3 11 2', 0.9555555555555556),
 ('5 2 3', 0.9555555555555556),
 ('5 5 2', 0.9555555555555556),
 ('5 7 2', 0.9555555555555556),
 ('5 19 2', 0.9555555555555556),
 ('6 7 3', 0.9555555555555556),
 ('6 14 3', 0.9555555555555556),
 ('7 11 3', 0.9555555555555556),
 ('7 16 2', 0.9555555555555556),
 ('8 1 1', 0.9555555555555556),
 ('9 3 2', 0.9555555555555556),
 ('9 9 2', 0.9555555555555556),
 ('9 10 1', 0.9555555555555556),
 ('9 10 3', 0.9555555555555556),
 ('10 16 2', 0.9555555555555556),
 ('11 14 3', 0.9555555555555556),
 ('11 17 2', 0.9555555555555556),
 ('12 2 2', 0.9555555555555556),
 ('12 7 2', 0.9555555555555556)